# Tribunal Superior Eleitoral

<img src="Imagens/TSE.jpg">

# Introdução

***
Neste artigo, irei demostrar os presidentes que tiveram maior número de votos durante os anos, utilizando dados disponíveis da [Base dos Dados](https://basedosdados.org/) em python. Os dados Utilizados correspondem a dados do [Tribunal Eleitoral](https://basedosdados.org/dataset/br-tse-eleicoes) das eleições de 1998 a 2018. \
A partir deste artigo, espera-se que você conheça quais presidentes ganharam e quase ganharam as eleições perante os anos eleitorais.

In [4]:
# instalando pacote da base dos dados
!pip install basedosdados 

Com o pacote instalado iremos carregar todos os pacotes que utilizaremos em nossa análise.

In [24]:
# Importando Pacotes
import basedosdados as bd # importa dados da base dos dados
import pandas as pd # manipulação de dados
import seaborn as sns # criação de gráficos
import matplotlib as plt # criações de gráficos

Feito a importação dos pacotes, agora iremos trazer os dados referentes aos candidatos perante os anos. Para isso, utilizaremos da função `read_sql` que permita que os dados possam ser trazidos via uma query em sql. Contudo, vamos adcionar a nossa query em uma várivel.

In [90]:
# Criando variável com a query em sql
query = """
Select 
    R.Ano, 
    C.nome as Candidatos, 
    R.turno,
    R.votos
from `basedosdados.br_tse_eleicoes.candidatos`as C
INNER JOIN `basedosdados.br_tse_eleicoes.resultados_candidato` as R
on  C.id_candidato_bd = R.id_candidato_bd
where R.cargo = "presidente"
Order by R.ano asc;
"""

Com a query montada, agora podemos carregar os dados via o comando `read_sql`.

In [91]:
# Carregando os dados
df = bd.read_sql(query, billing_project_id='carbide-legend-331517')

Downloading: 100%|██████████| 293/293 [00:00<00:00, 943.50rows/s]


Note que para o carregamento dos dados, tivemos que utilizar um `billing_project_id`. Para a obtenção deste código faz-se necessário um [projeto no google cloud](https://basedosdados.github.io/mais/access_data_bq/#primeiros-passos), conforme demonstra a Base dos Dados. Com os dados carregados, agora iremos verificar os dados disponíveis.

In [92]:
# Carregando as 5 primeiras linhas dos dados.
df.head()

,Ano,Candidatos,turno,votos
0,1998,José Maria De Almeida,1,202614
1,1998,Ivan Moacyr Da Frota,1,251275
2,1998,Ivan Moacyr Da Frota,1,251275
3,1998,Vasco Azevedo Neto,1,108969
4,1998,José Maria De Almeida,1,202614


In [93]:
# Número de linhas x colunas
linhas = df.shape[0] # linhas
colunas = df.shape[1]
print('Número de linhas = ',linhas)
print('Número de colunas = ', colunas)

Número de linhas =  293
Número de colunas =  4


In [94]:
# Verificando presença de valores duplicados
df.value_counts()

Ano   Candidatos                                     turno  votos   
2014  Luciana Krebs Genro                            1      1612186     9
      Aécio Neves Da Cunha                           1      34897211    6
      José Levy Fidelix Da Cruz                      1      446878      6
2018  Jair Messias Bolsonaro                         1      49277010    6
                                                     2      57797847    6
                                                                       ..
2002  Anthony Willian Garotinho Matheus De Oliveira  1      15180097    1
1998  Vasco Azevedo Neto                             1      108969      1
      Sergio Bueno                                   1      124546      1
      Fernando Henrique Cardoso                      1      35922534    1
2018  Álvaro Fernandes Dias                          1      859601      1
Length: 90, dtype: int64

In [95]:
# Removendo Valores duplicados
df = df.drop_duplicates()

In [96]:
# Número de linhas x colunas
linhas = df.shape[0] # linhas
colunas = df.shape[1]
print('Número de linhas = ',linhas)
print('Número de colunas = ', colunas)

Número de linhas =  90
Número de colunas =  4


Observe que dentro de nosso dataset temos 4 variáveis (Ano, Nome, Turno e Votos). Com os nossos dados corretamente tratados, iremos criar nossa visualização. Contudo, vamos ver quais anos tivemos eleição presencial dentro de nosso dataset.

In [97]:
Eleicoes = df['Ano'].unique()
print(Eleicoes)

[1998 2002 2006 2010 2014 2018]


In [85]:
x = 2020
if x in Eleicoes:
    print('ok')
else:
    print('Não ok')

Não ok


In [120]:
x = 2018
top = df.query('Ano == %s' % x)
turno02 = top.query('turno == 2')
turno02.loc[234,['Candidatos']]
turno02['votos'].max().inde

57797847

Agora que temos os anos com as eleções presidenciais, iremos criar uma função que traz um gráfico de barras com os top eleitores de acordo com o ano selecionado.

In [ ]:
df top_candidados_ano(ano):
    if ano in Eleicoes:
        top = df.query('Ano == %s' % ano)
        turno01 = top.query('turno == 1')
        turno02 = top.query('turno == 2')
        # Tema do gráfico
        
        plt.style.use("green")
        
        # Atribuindo um título ao gráfico
        
        plt.title('Eleição %s - Total de Votos' % ano)

        plt.pie(turno01['votos'], explode=explode, labels=  turno01['Candidatos'], , autopct='%1.1f%%', shadow=True, startangle=90)

        #Adiciona Legenda
        
        plt.legend(turno01['Candidatos'], bbox_to_anchor=(1.3, 1.3),loc='upper right')

        #Centraliza o gráfico
        plt.axis('equal')

        #Ajusta o espaçamento para evitar o recorte do rótulo
        
        plt.tight_layout()

        plt.show()
        
        if turno02.shape[0] == 2:
            percentual_ganhador = max(turno02['Candidatos'])/sum(turno02['Candidatos'])
            index = 
            ganhador = turno02.loc[index,['Candidatos']]
            return print
        else:
            
    else:
        return print("Neste ano não tivemos eleição presidencial")